<center><h1><font color="orange">Black Scholes Data Generator:</font></h1></center>

<h4>Imports:</h4>

In [1]:
import numpy as np
from math import exp, log, pi, sqrt
from scipy.stats import norm

<h4>Data Generator:</h4>

In [124]:
class BlackScholes:
    def input_generator(self,num_samples): 
        '''
        Generate random strike/spot, time (in days), rate, volatility (sigma), and dividend data to run 
        Black-Scholes on for n sampels
        '''
        S = np.random.uniform(0.00,500,num_samples) #Random Spot
        g = np.random.uniform(0.95,1.05,num_samples)
        K = S*g #Generate ranndom strike where we have have K/S between 0.95 and 1.0
        #K = np.random.uniform(0.00,110,num_samples) #Random Strike
        #KS = np.random.uniform(0.95,1.05,num_samples)  #The ratio of K/S
        #K = np.random.uniform(0.00,100,num_samples) #Random Strike -- check this
        #T = np.random.uniform(1,10*365,num_samples)   #Time In days
        #sigma = np.random.exponential(.10,num_samples)  #Random exponential sigma
        #rate = np.random.uniform(-0.2, 0.2, num_samples) # Annualized risk free rate
        #dividend = np.random.uniform(0.0, 0.2, num_samples) #dividend yield    
        #KS = np.random.uniform(0.95, 1.05, num_samples)                      # K/S
        T = np.random.uniform(0, 365*8, num_samples)                      # time until expiration in days
        rate = np.random.uniform(-0.2, 0.2, num_samples)                   # Annualized risk free rate
        sigma = np.random.uniform(0.10, 2.0, num_samples)               
        dividend = np.random.uniform(0.0, 0.2, num_samples)                # Dividend yield
        #X = np.asmatrix([KS,T,sigma,rate,dividend]).transpose()
        X = np.asmatrix([S,K,T,sigma,rate,dividend]).transpose()
        return X
    
    def getData(self,num_samples,write=False): 
        ''' 
        Get Random inputs X and prices Y
        '''
        X = self.input_generator(num_samples)
        Y = self.generate_prices(X)
        X = np.asarray(X)
        if write:
            with open('data.csv','w') as file:
                file.write('K/S,T,sigma,rate,dividend,v\n') 
                for i in range(X.shape[0]): 
                    file.write('{},{},{},{},{},{}\n'.format(X[i][0], X[i][1], X[i][2], X[i][3],X[i][4], Y[i]))
                file.close()
        return X,Y
    
    def generate_prices(self,X): 
        '''
        Vectorization to call the vanilla call price from the input
        '''
        return np.apply_along_axis(self.vanilla_call_price, axis=1, arr=X)
    

    def vanilla_call_price(self,X): 
        '''
        Calculate Vanialla Call Price
        '''

        #Get Parameters from X
        S = X[0,0]
        K = X[0,1]
        T = X[0,2]
        sigma = X[0,3]
        r = X[0,4]
        d = X[0,5]
        

        #Black Scholes Formula Calculation
        #edt = exp((-d * T) / 365)
        #ert = exp((-r * T) / 365)
        #sigmaTsq = sqrt(T / 365) * sigma
        #d1 = (log(edt/(K/S)) + (r + .5 * (sigma ** 2)) * T / 365) / sigmaTsq
        #d1 = (log(edt/KS) + (r + .5 * (sigma ** 2)) * T / 365) / sigmaTsq
        #d2 = d1 - sigmaTsq
        #Nd1 = (1 + ert*(d1 / sqrt(2))) / 2
        #Nd2 = (1 + ert*(d2 / sqrt(2))) / 2

        #v = edt*Nd1 - (K/S) * ert * Nd2
        #v = edt*Nd1 - KS * ert * Nd2
        d1 = (log(float(S)/K)+((r-d)+sigma*sigma/2.)*T/365)/(sigma*sqrt(T))
        d2 = d1-sigma*sqrt(T)
        return S*exp(-d*T/365)*norm.cdf(d1)-K*exp(-r*T/365)*norm.cdf(d2)
        
        
        
        #return v
    
    #def d_j(self,j, S, K, r, v, T):
    #    return (log(S/K) + (r + ((-1)**(j-1))*0.5*v*v)*T)/(v*(T**0.5))
    
    #def vanilla_call_price(self,X):
    #    S = X[0,0]
    #    K = X[0,1]
    #    T = X[0,2]
    #    sigma = X[0,3]
    #    r = 0
    #    return S * norm.cdf(self.d_j(1, S, K, r, sigma, T)) - \
    #        K*exp(-r*T) * norm.cdf(self.d_j(2, S, K, r, sigma, T))
    


In [125]:
b = BlackScholes()
X,Y = b.getData(20000)

In [127]:
testX,testY = BlackScholes().getData(5000)

<h4>Models:</h4>

In [128]:
from keras.models import Sequential
from keras.layers import Dense

In [129]:
model = Sequential()
model.add(Dense(64,input_dim=X.shape[1],activation='tanh'))
#model.add(Dense(32, activation='tanh'))
#model.add(Dense(8,activation='tanh'))
model.add(Dense(1,activation='linear'))
model.compile(optimizer='adam',loss='mse',metrics=['mse'])

In [131]:
model.fit(X,Y,epochs=100, batch_size=32)

Epoch 1/100
20000/20000 [==============================] - 0s 23us/step - loss: 929.8000 - mean_squared_error: 929.8000
Epoch 2/100
20000/20000 [==============================] - 0s 22us/step - loss: 891.6509 - mean_squared_error: 891.6509
Epoch 3/100
20000/20000 [==============================] - 0s 22us/step - loss: 858.4084 - mean_squared_error: 858.4084
Epoch 4/100
20000/20000 [==============================] - 0s 22us/step - loss: 818.8868 - mean_squared_error: 818.8868
Epoch 5/100
20000/20000 [==============================] - 0s 23us/step - loss: 806.6795 - mean_squared_error: 806.6795
Epoch 6/100
20000/20000 [==============================] - 0s 22us/step - loss: 759.2286 - mean_squared_error: 759.2286
Epoch 7/100
20000/20000 [==============================] - 0s 22us/step - loss: 723.8397 - mean_squared_error: 723.8397
Epoch 8/100
20000/20000 [==============================] - 0s 23us/step - loss: 691.4145 - mean_squared_error: 691.4145
Epoch 9/100
20000/20000 [===============

20000/20000 [==============================] - 0s 24us/step - loss: 329.4639 - mean_squared_error: 329.4639
Epoch 69/100
20000/20000 [==============================] - 0s 24us/step - loss: 344.1618 - mean_squared_error: 344.1618
Epoch 70/100
20000/20000 [==============================] - 0s 24us/step - loss: 341.2300 - mean_squared_error: 341.2300
Epoch 71/100
20000/20000 [==============================] - 0s 24us/step - loss: 359.0397 - mean_squared_error: 359.0397
Epoch 72/100
20000/20000 [==============================] - 0s 24us/step - loss: 335.3946 - mean_squared_error: 335.3946
Epoch 73/100
20000/20000 [==============================] - 0s 24us/step - loss: 362.1892 - mean_squared_error: 362.1892
Epoch 74/100
20000/20000 [==============================] - 0s 24us/step - loss: 335.0117 - mean_squared_error: 335.0117
Epoch 75/100
20000/20000 [==============================] - 0s 24us/step - loss: 327.3455 - mean_squared_error: 327.3455
Epoch 76/100
20000/20000 [===================

In [132]:
preds = model.predict(testX)

In [134]:
preds

array([[151.57826 ],
       [ 65.21415 ],
       [ 28.403301],
       ...,
       [ 98.86843 ],
       [213.56342 ],
       [ 61.691353]], dtype=float32)

In [135]:
testY

array([159.0415273 ,  60.93283996,  29.3640982 , ...,  76.69543475,
       201.30210005,  29.67204858])

In [136]:
model.evaluate(testX,testY)

5000/5000 [==============================] - 0s 47us/step


[308.02887568359375, 308.02887568359375]